In [144]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

import pandas as pd
import numpy as np

In [145]:
#### 데이터 로드
df_ciminal = pd.read_csv('./data/pp_01_housingdata.csv')
x_np = df_ciminal.iloc[:, :12].values
x_np = (x_np - np.mean(x_np, axis=0)) / np.std(x_np, axis=0) # 표준화
y_np = df_ciminal.iloc[:, 13].values.reshape(-1, 1)

#### 셔플
z = np.arange(x_np.shape[0])
np.random.shuffle(z)
x_np = x_np[z]
y_np = y_np[z]

print(x_np.shape)
print(y_np.shape)


#### train, test 분할
cut_idx = int(len(x_np)*0.8)

train_x  = x_np[:cut_idx]
train_y  = y_np[:cut_idx]
test_x  = x_np[cut_idx:]
test_y  = y_np[cut_idx:]

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

#### 하이퍼 파라미터
batch_size = 2
learning_rate = 0.001
num_epoch = 100



(275, 12)
(275, 1)
(220, 12)
(220, 1)
(55, 12)
(55, 1)


In [146]:
class cvt_to_torch_dataset(Dataset):
    def __init__(self, x_data, y_data, transform = None):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
                
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [147]:
train_dataset = cvt_to_torch_dataset(train_x, train_y)
test_dataset = cvt_to_torch_dataset(test_x, test_y)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [148]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(12, 256)  # input
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, 8)
        self.fc7 = nn.Linear(8, 1) # ouput
        

    def forward(self, x):
        # x = x.view(-1, 12)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = self.fc7(x)
        return x

In [149]:
#### 모델 초기화
model = SimpleANN()

#### 손실 함수와 최적화 알고리즘 정의
loss_fn = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### 모델 학습
for epoch in range(num_epoch):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 출력
        running_loss += loss.item()
        # if i % 1 == 0:  # 매 100 미니배치마다 출력
        #     print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
        #     running_loss = 0.0
            
    if epoch % 5 == 0:
        print(f'[Epoch {epoch + 1}] loss: {running_loss / 100:.3f}')

print('Finished Training')
print('batch_size :', batch_size)
print('learning_rate :', learning_rate)



[Epoch 1] loss: 288.270
[Epoch 6] loss: 13.442
[Epoch 11] loss: 9.726
[Epoch 16] loss: 8.147
[Epoch 21] loss: 6.735
[Epoch 26] loss: 5.464
[Epoch 31] loss: 4.318
[Epoch 36] loss: 5.497
[Epoch 41] loss: 3.950
[Epoch 46] loss: 4.699
[Epoch 51] loss: 4.509
[Epoch 56] loss: 3.264
[Epoch 61] loss: 4.949
[Epoch 66] loss: 3.323
[Epoch 71] loss: 3.125
[Epoch 76] loss: 3.064
[Epoch 81] loss: 3.598
[Epoch 86] loss: 2.554
[Epoch 91] loss: 2.903
[Epoch 96] loss: 1.996
Finished Training
batch_size : 2
learning_rate : 0.001


In [ ]:
#### 평가
count_y = 0
y_sum = 0
loss_list = []
with torch.no_grad():
    for data in testloader: # 배치 단위 반복
        x_data, y_data = data
        outputs = model(x_data)
        loss = loss_fn(outputs, y_data)
        loss_list.append(loss)
        
        #### 
        count_y += y_data.shape[0] # 개수
        y_sum += y_data.numpy().sum() # y 총 합
        

loss_np = np.array(loss_list)
mean_loss = loss_np.mean()
y_mean = y_sum / count_y

print('count_y :', count_y)
print('y_sum :', y_sum)

print('y_mean :', y_mean)
print('mean_loss : ', mean_loss)
print('loss_per : ', mean_loss / y_mean) # 오차율

count_y : 55
y_sum : 1258.300012588501
y_mean : 22.878182047063653
mean_loss :  6.65978
loss_per :  0.2910974312461571


In [151]:
#### 저장
# torch.save(model.state_dict(), "model.pth")
# print("Saved PyTorch Model State to model.pth")

#### 로드
# model = SimpleANN()
# model.load_state_dict(torch.load("model.pth"))